# Flow-based Generative Model
[Video for Flow-based Generative Model](https://www.youtube.com/watch?v=uXY18nzdSsM)

## Introduction
### Math background
**Denotation**:
+ $\mathbf{x}$: Real data, such as image. It is a vector/matrix/tensor.
+ $\mathbf{z}$: Latent variable, and also serve as input variables in generation task
+ $\pi(z)$: The probability of producing $z$, i.e., the probability density function for $z$. In general, $\pi(z)$ is selected as normal distribution.
+ $p(x)$: The probability density function for $x$
+ $f$: The generative model who takes $z$ as input and produces $x$, i.e., $\mathbf{z} = f(\mathbf{x})$
+ Jacobian determinant (the function maps from one dimension to the same dimension, i.e., $n = m$)
    + The product of Jacobian determinant for a function and its inverse equals to 1, i.e., $\text{det}(\mathbf{J}_f) \text{det}(\mathbf{J}_{f^{-1}}) = 1$
+ $\text{det}(\mathbf{x})$: The physical meaning of determinant is the volume of object that its row/column vectors make up: <img src="images/determinant.png" width="350"/>




### Idea


For 1 dimension, the relationship between $\pi(z)$ and $p(x)$:
$$
\pi(z)|\Delta{z}| = p(x)|\Delta{x}| \\
\pi(z) = p(x)\left|\frac{\Delta{x}}{\Delta{z}}\right| \\
\pi(z) = p(x) \left|\frac{\text{d}x}{\text{d}z}\right|
$$
For 2 dimension, the relation becomes:
$$
\pi(z)|\Delta{z_1}\Delta{z_2}| = p(x) \left|\text{det}\begin{bmatrix}
\Delta x_{11} & \Delta x_{21} \\
\Delta x_{12} & \Delta x_{22} \\
\end{bmatrix}\right| \\
\pi(z) = p(x) \left|\text{det}\begin{bmatrix}
\frac{\partial x_1}{\partial z_1} & \frac{\partial x_2}{\partial z_1} \\
\frac{\partial x_1}{\partial z_2} & \frac{\partial x_2}{\partial z_2} \\
\end{bmatrix}\right| \\
\pi(z) = p(x)|\text{det}(\mathbf{J}_f)| \\
\pi(z)\frac{1}{|\text{det}(\mathbf{J}_f)|} = p(x) \\ 
\pi(z)|\text{det}(\mathbf{J}_{f^{-1}})| = p(x)  
$$

**Relationship between $\pi(z)$ and $p(x)$**:
$$
\pi(z) = p(x)|\text{det}(\mathbf{J}_f)| \\
\pi(z)|\text{det}(\mathbf{J}_{f^{-1}})| = p(x) 
$$



In generative model, we want the probability of real data to be large. Therefore we need to maximize $p(x)$ where $x$ is sampled from real data. the target function is:
$$
F = \text{arg }\underset{G}{\text{max}}{\sum_i^m{\log{p_G(x^i)}}},
$$
where $G$ is the generative function, as well as $f$ we discussed above. 

Based on the aforementioned discussion, $p_G(x^{i})$ can be replaced with:
$$
p_G(x^{i}) = \pi(z^i)|\text{det}(\mathbf{J}_{G^{-1}})|
$$

Therefore, the target function becomes:
\begin{aligned}
F & = \text{arg }\underset{G}{\text{max}}{\sum_i^m{\log{ \pi(z^i)|\text{det}(\mathbf{J}_{G^{-1}})|  }}} \\
& = \text{arg }\underset{G}{\text{max}}{\sum_i^m({\log{ \pi(z^i) + \log{|\text{det}(\mathbf{J}_{G^{-1}})|})  }}}
\end{aligned}

$z^i$ is the input for $G$, and thus the output of $G^{-1}$ taking $x^i$ as input. The equation above can be further transformed into:


**Target function:**
$$
F = \text{arg }\underset{G}{\text{max}}{\sum_i^m({\log{ \pi(G^{-1}(x^i)) + \log{|\text{det}(\mathbf{J}_{G^{-1}})|})  }}}
$$

What does this target function means? Since $\pi(\mathbf{x})$ is normal distribution (we assume latent variables follow normal distribution), this target function depends only on $G^{-1}$ and $\mathbf{x}$, i.e., the inverse of generative model and input real data. Therefore, we use real data to train an inverse of generative model by optimizing target function value.


To compute objective function fast, we need to find an efficient algorithm to calculate $G^{-1}(\mathbf{x})$ and $\mathbf{J}_{G^{-1}}$. One of solutions is coupling layer.

#### Coupling Layer
The overview of coupling layer is:
<img src="images/coupling_layer_forward.png" width="350"/>


The coupling layer takes $\mathbf{z}$ as input and outputs $\mathbf{x}$. First, we split $\mathbf{z}$ into 2 parts. Then we copy the first part to $\mathbf{x}$ and computes the second part of $\mathbf{x}$.

**1. Coupling layer is friendly to calculation of $G^{-1}$.** The figure below shows the inverse of coupling layer.
<img src="images/coupling_layer_inverse.png" width="350"/>

**2. Coupling layer is friendly to calculation of $\mathbf{J}_{G^{-1}}$.** The reason that $F$ and $H$ can be any functions is the derivative of second part of $\mathbf{x}$ over first part of $\mathbf{z}$ is removed from the calculation of $\mathbf{J}_{G^{-1}}$. 

<img src="images/coupling_layer_determinant.png" width="350"/>


What if $F$ and $H$ is invertable? It happens rarely, since it requires a determinant of matrix to be zero which can be arbitrary values.

**3. Randomly split $z$ to avoid the last image has the exact values from noise.**
<img src="images/coupling_layer_random.png" width="350"/>


**Coupling layer is stacked to form a flow**:
<img src="images/flow.png" width="350"/>